In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df= pd.read_csv("./전처리/df_stock1.csv")
df_stock = df.drop(['Unnamed: 0','Code'],axis = 1)

# 1월3일의 결과를 1월 2일의 기사를 토대로 예측해야 하므로 한칸 위로
df_stock['target']=df_stock['label'].shift(-1)
# shift가 발생하면서 1월2일의 라벨들이 12월 30이롤 이동하기 때문에 삭제
check = (df_stock['Date'] == '2019-12-30')
# ~ 기호를 이용하여, 해당하지 않은 데이터만 남긴다
df_stock = df_stock[~check]

# 라벨인코딩을 통해서, 해당 타겟 값을 바꿔준다.
encoder = LabelEncoder()
encoder.fit(df_stock['target'])
labels = encoder.transform(df_stock['target'])
df_stock['target'] = labels

In [5]:
# main - feature 500

df_news = pd.read_csv('./전처리/TF-IDF/main_tf-idf_500.csv')
df_stock['Date'] = df_stock['Date'].str.replace('-','.')
df_stock['id'] = df_stock['Name']+df_stock['Date']
df_news['id'] = df_news['Company']+df_news['Day']
total_df = pd.merge(df_stock, df_news, on = 'id').drop(['Name','Date','id','Company','Day','Unnamed: 0','label'],axis=1)
total_df = total_df.dropna()

setup_clf = setup(data=total_df, target='target', session_id = 1, use_gpu = True)
top5_main_500 = compare_models(sort='Accuracy', n_select=5,exclude=['catboost','gbc','svm','ridge'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.6576,0.5265,0.0170,0.3953,0.0325,0.0043,0.0136,0.5620
rf,Random Forest Classifier,0.6572,0.5266,0.0120,0.3283,0.0231,0.0003,-0.0007,3.2050
et,Extra Trees Classifier,0.6558,0.5230,0.0105,0.2934,0.0202,-0.0034,-0.0111,2.4080
ada,Ada Boost Classifier,0.6498,0.5152,0.0436,0.3632,0.0776,0.0056,0.0105,21.0970
lightgbm,Light Gradient Boosting Machine,0.6496,0.5166,0.0396,0.3517,0.0711,0.0029,0.0055,1.5500
lda,Linear Discriminant Analysis,0.6447,0.5265,0.0760,0.3799,0.1266,0.0156,0.0234,1.3520
knn,K Neighbors Classifier,0.6173,0.5434,0.2410,0.3952,0.2992,0.0569,0.0602,8.2790
qda,Quadratic Discriminant Analysis,0.5965,0.5333,0.2799,0.3740,0.3198,0.0415,0.0425,0.9760
dt,Decision Tree Classifier,0.5497,0.5027,0.3568,0.3430,0.3496,0.0056,0.0057,6.5220
nb,Naive Bayes,0.5443,0.5201,0.4170,0.3544,0.3830,0.0257,0.0259,0.1150


In [6]:
# main - feature 2000

df_news = pd.read_csv('./전처리/TF-IDF/main_tf-idf_2000.csv')
df_stock['Date'] = df_stock['Date'].str.replace('-','.')
df_stock['id'] = df_stock['Name']+df_stock['Date']
df_news['id'] = df_news['Company']+df_news['Day']
total_df = pd.merge(df_stock, df_news, on = 'id').drop(['Name','Date','id','Company','Day','Unnamed: 0','label'],axis=1)
total_df = total_df.dropna()

setup_clf = setup(data=total_df, target='target', session_id = 1, use_gpu = True)
top5_main_2000 = compare_models(sort='Accuracy', n_select=5,exclude=['catboost','gbc','svm','ridge'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.6580,0.5345,0.0137,0.3719,0.0264,0.0030,0.0087,5.9940
rf,Random Forest Classifier,0.6561,0.5290,0.0103,0.2904,0.0198,-0.0029,-0.0104,5.6460
qda,Quadratic Discriminant Analysis,0.6542,0.5431,0.0606,0.4318,0.1060,0.0245,0.0433,10.6040
lr,Logistic Regression,0.6538,0.5475,0.0410,0.3973,0.0742,0.0117,0.0235,1.9070
lightgbm,Light Gradient Boosting Machine,0.6536,0.5266,0.0543,0.4157,0.0960,0.0196,0.0352,10.3040
ada,Ada Boost Classifier,0.6431,0.5145,0.0596,0.3471,0.1017,0.0027,0.0043,51.6140
knn,K Neighbors Classifier,0.6258,0.5616,0.2715,0.4205,0.3298,0.0866,0.0907,31.9560
lda,Linear Discriminant Analysis,0.6105,0.5401,0.2492,0.3854,0.3026,0.0494,0.0517,10.1280
dt,Decision Tree Classifier,0.5503,0.5034,0.3580,0.3433,0.3504,0.0067,0.0068,21.6950
nb,Naive Bayes,0.5302,0.5293,0.4856,0.3583,0.4122,0.0358,0.0369,0.4900


In [2]:
# head - feature 500
df_news = pd.read_csv('./전처리/TF-IDF/head_tf-idf_500.csv')
df_stock['Date'] = df_stock['Date'].str.replace('-','.')
df_stock['id'] = df_stock['Name']+df_stock['Date']
df_news['id'] = df_news['Company']+df_news['Day']
total_df = pd.merge(df_stock, df_news, on = 'id').drop(['Name','Date','id','Company','Day','Unnamed: 0','label'],axis=1)
total_df = total_df.dropna()
target = total_df['target']
test = total_df.drop(['target'], axis=1)

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(test, target, test_size=0.3, shuffle=False, random_state=100)

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)
pred = lr_clf.predict(X_test)
print(accuracy_score(y_test, pred))

0.6465179378956889


In [5]:
import eli5
from eli5.sklearn  import PermutationImportance

perm1 = PermutationImportance(lr_clf, scoring = "f1", random_state = 42).fit(X_test, y_test)
eli5.show_weights(perm1, top = 80, feature_names = X_test.columns.tolist())

Weight,Feature
0.0027 ± 0.0033,올해
0.0023 ± 0.0024,증시
0.0023 ± 0.0012,개선
0.0021 ± 0.0024,경제
0.0019 ± 0.0012,주년
0.0018 ± 0.0008,준비
0.0018 ± 0.0023,미래
0.0017 ± 0.0020,약보합
0.0017 ± 0.0019,코스닥
0.0016 ± 0.0010,관련


In [6]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score

print("오차 행렬\n", confusion_matrix(y_test, pred))
print("정확도 : ", accuracy_score(y_test, pred))
print("정밀도 : ", precision_score(y_test, pred))
print("재현율 : ", recall_score(y_test, pred))
print("f1 : ", f1_score(y_test,pred))
print("roc : ", roc_auc_score(y_test,pred))

오차 행렬
 [[4218  145]
 [2200   71]]
정확도 :  0.6465179378956889
정밀도 :  0.3287037037037037
재현율 :  0.03126376045794804
f1 :  0.057096903900281465
roc :  0.49901487358217134
